# Currents

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import support as sp

from tqdm import tqdm_notebook
import tqdm

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
directory = "/home/aortiza/Data/Dimers/FrequencySweep_3.9/"

In [3]:
index = pd.read_csv(os.path.join(directory,"index.dat"),names = ["file", "height", "tilt"], sep=" ")

In [4]:
from multiprocessing import Pool

In [5]:
fmax = 7.5*ureg.Hz # 7.5Hz
dt = 15*ureg.sec # 15sec
df = 0.125*ureg.Hz

ratio = df/dt
total_time = fmax/ratio

f = st.Template("$ratio*ceil(time/$dt)*$dt").substitute(
    dt = dt.to(ureg.us).magnitude,
    ratio = ratio.to((ureg.MHz/ureg.us)).magnitude) * ureg.MHz

frequency = lambda time: (ratio.to((ureg.MHz/ureg.us))*(np.ceil(time.to(ureg.us)/dt.to(ureg.us))*dt)).to(ureg.Hz)
tilt_angle = np.concatenate([np.arange(0,30.5,0.5)])

In [6]:
def subset(times, dt, rnge):
    """ Defines a repeated range of times. It returns the same range for every value of frequency"""
    fun = lambda times: (np.mod(times[:],dt))/dt
    thresh1 = (min(rnge))/dt
    thresh2 = (max(rnge))/dt
    return (fun(times)>thresh1) & (fun(times)<thresh2)

In [7]:
def differentiated_velocity(name):
    ## Lazy open the trajectory
    trj_read = mgc.trj_lazyread(os.path.join(directory,name.file+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])

    ## Define a subset
    timestep = 1e-4*ureg.s
    frames = np.array(list(trj_read.T.keys()))
    times = frames*timestep
    readthis = subset(times, dt, [7*ureg.s,14*ureg.s])

    ## Load Trajectory
    trj = trj_read.read_trj(np.where(readthis))
    bounds = trj_read.get_bounds(slice(0,1,1))
    trj = trj.filter(["x","y","z"])
    trj["time"] = trj.index.get_level_values("frame")*timestep.to(ureg.s).magnitude
    trj["frequency"] = frequency(trj.time.values*ureg.s).to(ureg.Hz).magnitude

    ## Calculate Velocity Array
    velocities = sp.calculate_velocities(trj, bounds)

    ## Calculate Differentiated Velocity
    velocities["plane"] = np.NaN
    velocities.loc[trj.z>0,"plane"] = "up"
    velocities.loc[trj.z<0,"plane"] = "down"
    vupdown = velocities.groupby(["plane","frequency"]).mean().filter(["x","y"])
    vupdown.to_csv(os.path.join(directory,name.file+"_vupdown.dat"), sep="\t")
    
    return vupdown

In [8]:
%%time
if __name__ ==  '__main__': 
    num_processors = 2
    p=Pool(processes = num_processors)
    diff_vel = list(tqdm.tqdm_notebook(
            p.imap(differentiated_velocity, 
                   [name for i,name in index.iterrows()]), 
            total=len(index)))


CPU times: user 2.79 s, sys: 1.08 s, total: 3.87 s
Wall time: 1h 24min 50s


In [9]:
diff_vel_pd = pd.concat(diff_vel, keys = index.index, names=["file"])

In [10]:
diff_vel_pd.to_csv(os.path.join(directory,"differentiated_velocities.dat"), sep = "\t")

# Currents h = 3.5

In [11]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import support as sp

from tqdm import tqdm_notebook
import tqdm

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [12]:
directory = "/home/aortiza/Data/Dimers/FrequencySweep_3.5/"

In [13]:
index = pd.read_csv(os.path.join(directory,"index.dat"),names = ["file", "height", "tilt"], sep=" ")

In [14]:
from multiprocessing import Pool

In [15]:
fmax = 7.5*ureg.Hz # 7.5Hz
dt = 15*ureg.sec # 15sec
df = 0.125*ureg.Hz

ratio = df/dt
total_time = fmax/ratio

f = st.Template("$ratio*ceil(time/$dt)*$dt").substitute(
    dt = dt.to(ureg.us).magnitude,
    ratio = ratio.to((ureg.MHz/ureg.us)).magnitude) * ureg.MHz

frequency = lambda time: (ratio.to((ureg.MHz/ureg.us))*(np.ceil(time.to(ureg.us)/dt.to(ureg.us))*dt)).to(ureg.Hz)
tilt_angle = np.concatenate([np.arange(0,30.5,0.5)])

In [16]:
def subset(times, dt, rnge):
    """ Defines a repeated range of times. It returns the same range for every value of frequency"""
    fun = lambda times: (np.mod(times[:],dt))/dt
    thresh1 = (min(rnge))/dt
    thresh2 = (max(rnge))/dt
    return (fun(times)>thresh1) & (fun(times)<thresh2)

In [17]:
def differentiated_velocity(name):
    ## Lazy open the trajectory
    trj_read = mgc.trj_lazyread(os.path.join(directory,name.file+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])

    ## Define a subset
    timestep = 1e-4*ureg.s
    frames = np.array(list(trj_read.T.keys()))
    times = frames*timestep
    readthis = subset(times, dt, [7*ureg.s,14*ureg.s])

    ## Load Trajectory
    trj = trj_read.read_trj(np.where(readthis))
    bounds = trj_read.get_bounds(slice(0,1,1))
    trj = trj.filter(["x","y","z"])
    trj["time"] = trj.index.get_level_values("frame")*timestep.to(ureg.s).magnitude
    trj["frequency"] = frequency(trj.time.values*ureg.s).to(ureg.Hz).magnitude

    ## Calculate Velocity Array
    velocities = sp.calculate_velocities(trj, bounds)

    ## Calculate Differentiated Velocity
    velocities["plane"] = np.NaN
    velocities.loc[trj.z>0,"plane"] = "up"
    velocities.loc[trj.z<0,"plane"] = "down"
    vupdown = velocities.groupby(["plane","frequency"]).mean().filter(["x","y"])
    vupdown.to_csv(os.path.join(directory,name.file+"_vupdown.dat"), sep="\t")
    
    return vupdown

In [18]:
%%time
if __name__ ==  '__main__': 
    num_processors = 2
    p=Pool(processes = num_processors)
    diff_vel = list(tqdm.tqdm_notebook(
            p.imap(differentiated_velocity, 
                   [name for i,name in index.iterrows()]), 
            total=len(index)))


CPU times: user 4.89 s, sys: 1.93 s, total: 6.82 s
Wall time: 1h 23min 22s


In [19]:
diff_vel_pd = pd.concat(diff_vel, keys = index.index, names=["file"])

In [20]:
diff_vel_pd.to_csv(os.path.join(directory,"differentiated_velocities.dat"), sep = "\t")